In [1]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, SimpleRNN, Dropout, RepeatVector
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping

from termcolor import colored

In [2]:
np.random.seed(52)
all_char = '0123456789+'

num_feature = len(all_char)

char_to_idx = dict((c,i) for i,c in enumerate(all_char))
idx_to_char = dict((i,c) for i,c in enumerate(all_char))

In [3]:
def generate_data():
    first = np.random.randint(0,100)
    second = np.random.randint(0,100)
    data = str(first)+'+'+str(second)
    label = str(first+second)
    return data,label
generate_data()    

('28+11', '39')

Creating  the model

In [4]:
hidden_units = 128
max_time_steps = 5

model = Sequential([
    SimpleRNN(hidden_units, input_shape = (None, num_feature)),
    RepeatVector(max_time_steps),
    SimpleRNN(hidden_units, return_sequences = True),
    TimeDistributed(Dense(num_feature, activation = 'softmax'))
])
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['acc']

)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 128)               17920     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 5, 128)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 5, 128)            32896     
_________________________________________________________________
time_distributed (TimeDistri (None, 5, 11)             1419      
Total params: 52,235
Trainable params: 52,235
Non-trainable params: 0
_________________________________________________________________


In [5]:
def vectorize_example(example, label):
    x = np.zeros((max_time_steps, num_feature))
    y = np.zeros((max_time_steps, num_feature))
    
    diff_x = max_time_steps - len(example)
    diff_y = max_time_steps - len(label)
    
    for i,c in enumerate(example):
        x[i+diff_x,char_to_idx[c]] = 1
    for i in range(diff_x):
        x[i, char_to_idx['0']] = 1
    
    for i,c in enumerate(label):
        y[i+diff_y,char_to_idx[c]] = 1
    
    for i in range(diff_y):
        y[i, char_to_idx['0']] = 1
    return x,y

e,l = generate_data()
e_v, l_v = vectorize_example(e,l)
print(e+' = '+l)

13+23 = 36


In [6]:
def devectorize(example):
    res = [idx_to_char[np.argmax(vec)] for vec in example ]
    return ''.join(res)

print(devectorize(l_v))

00036


Create dataset

In [7]:
def Create_dataset(num_data):
    x = np.zeros((num_data, max_time_steps, num_feature))
    y = np.zeros((num_data, max_time_steps, num_feature))
    
    for i in range(num_data):
        e, l =generate_data()
        e_v, l_v = vectorize_example(e, l)
        x[i] = e_v
        y[i] = l_v
    return x,y

write callbacks

In [8]:
x,y = Create_dataset(num_data=2000)
l_cb = LambdaCallback(
    on_epoch_end= lambda e, l : print('{:0.2f}'.format(l['val_acc']),end='_')
)
es_cb = EarlyStopping(monitor='val_loss', patience=10)

model.fit(
    x,y,epochs=500, batch_size=256, validation_split=0.2,verbose=False,
    callbacks=[l_cb,es_cb]
)

0.57_0.61_0.62_0.63_0.63_0.64_0.64_0.64_0.64_0.65_0.65_0.65_0.65_0.66_0.66_0.68_0.67_0.67_0.69_0.70_0.70_0.71_0.71_0.72_0.73_0.72_0.72_0.72_0.73_0.73_0.72_0.74_0.74_0.75_0.75_0.75_0.76_0.76_0.76_0.76_0.77_0.76_0.77_0.78_0.78_0.79_0.79_0.79_0.80_0.80_0.80_0.80_0.82_0.82_0.82_0.82_0.83_0.83_0.84_0.84_0.85_0.86_0.85_0.86_0.87_0.88_0.87_0.88_0.88_0.88_0.89_0.89_0.89_0.90_0.90_0.90_0.89_0.90_0.90_0.91_0.90_0.90_0.91_0.91_0.91_0.91_0.91_0.91_0.91_0.91_0.91_0.92_0.92_0.92_0.92_0.92_0.92_0.92_0.93_0.93_0.92_0.92_0.93_0.93_0.93_0.93_0.93_0.94_0.94_0.93_0.93_0.93_0.94_0.94_0.94_0.94_0.94_0.94_0.94_0.94_0.94_0.93_0.94_0.94_0.95_0.94_0.94_0.94_0.95_0.95_0.94_0.95_0.94_0.95_0.94_0.95_0.95_0.95_0.94_0.95_0.95_0.95_0.95_0.95_0.95_0.95_0.95_0.95_0.95_0.95_0.95_0.95_0.95_0.96_0.95_0.95_0.95_0.95_0.95_0.95_0.95_0.95_0.95_0.95_0.95_0.95_0.95_0.96_0.95_0.95_0.95_0.95_0.95_0.95_0.95_0.96_0.96_0.95_0.95_0.95_0.95_0.95_0.96_0.95_0.96_0.96_0.95_0.95_0.95_0.95_

Testing 

In [9]:
x_test,y_test = Create_dataset(num_data=20)

preds = model.predict(x_test)

for i,pred in enumerate(preds):
    ans = devectorize(y_test[i])
    prediction = devectorize(pred)
    
    col = 'green'
    
    if ans!=prediction:
        col = 'red'
    out = 'Input: '+devectorize(x_test[i])+' ans = '+ans+' prediction = '+prediction
    print(colored(out,col))
        

Input: 77+30 ans = 00107 prediction = 00107
Input: 60+14 ans = 00074 prediction = 00074
Input: 61+16 ans = 00077 prediction = 00077
Input: 58+10 ans = 00068 prediction = 00068
Input: 24+58 ans = 00082 prediction = 00082
Input: 60+79 ans = 00139 prediction = 00139
Input: 04+11 ans = 00015 prediction = 00025
Input: 77+48 ans = 00125 prediction = 00125
Input: 92+50 ans = 00142 prediction = 00142
Input: 59+66 ans = 00125 prediction = 00125
Input: 50+11 ans = 00061 prediction = 00061
Input: 88+85 ans = 00173 prediction = 00173
Input: 49+17 ans = 00066 prediction = 00066
Input: 63+23 ans = 00086 prediction = 00086
Input: 20+38 ans = 00058 prediction = 00058
Input: 004+5 ans = 00009 prediction = 00019
Input: 22+98 ans = 00120 prediction = 00120
Input: 04+76 ans = 00080 prediction = 00080
Input: 02+21 ans = 00023 prediction = 00023
Input: 63+59 ans = 00122 prediction = 00122
